# Divide data into test/training sets

In [ ]:
from pathlib import Path
import pickle

In [ ]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
export_fp = Path(config['project_directory']) / 'Calibration'
cycleatl_fp = Path(config['project_directory']) / 'CycleAtlanta'
matching_fp = Path(config['project_directory']) / 'Map_Matching'
network_fp = Path(config['project_directory']) / 'Network'
if export_fp.exists() == False:
    export_fp.mkdir()

# Import Matched Trace Data
Dictionary Containing Origins and Destinations, Matches, Shortest Paths

In [ ]:
#export for impedance calibration
with (matching_fp/'ready_for_calibration.pkl').open('rb') as fh:
    match_results = pickle.load(fh)
with (cycleatl_fp/'gps_traces/trips_3.pkl').open('rb') as fh:
    reduced_trip_set = pickle.load(fh)
filter_trips = set(reduced_trip_set['tripid'].tolist())

In [ ]:
match_results = {key:item for key, item in match_results.items() if key in filter_trips}
print(len(match_results))

## Split the data into training and testing set
For starters, lets select 500 random trips to go in each

In [ ]:
import random
def get_random_keys(dictionary,num_keys):
    random_keys = set()
    while len(random_keys) < num_keys:
        random_key =  random.choice(list(dictionary.keys()))
        if isinstance(dictionary.get(random_key),str):
            continue
        if random_key not in random_keys:
            random_keys.add(random_key)
    return list(random_keys)

In [ ]:
total_trips = 500
random_keys = get_random_keys(match_results,total_trips)

middle_point = int(total_trips/2)

train_set = random_keys[0:middle_point]
test_set = random_keys[middle_point:]

In [ ]:
#subset accordingly
train_set = {key:item for key, item in match_results.items() if key in train_set}
test_set = {key:item for key, item in match_results.items() if key in test_set}

In [ ]:
with (fp/'gps_traces/test_set.pkl').open('wb') as fh:
    pickle.dump(train_set,fh)
with (fp/'gps_traces/train_set.pkl').open('wb') as fh:
    pickle.dump(test_set,fh)